In [58]:
import pandas as pd

# # DOMA
# nodes_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
# roads_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\Roads_processed.csv"
# bridges_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\df_bridges_data.csv"
# survey_path = r"C:\Users\Doktor\Documents\GitHub\Bridges\Data\surveys\Survey.csv"

# ROBOTA
nodes_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Nodes_processed.csv"
roads_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\Roads_processed.csv"
bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_data.csv"
survey_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\surveys\Survey.csv"

# Load the CSV data into a DataFrame
df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')
df_survey = pd.read_csv(survey_path, encoding='utf-8', delimiter=',')

C:\Users\relia\AppData\Local\Temp\ipykernel_5380\4245361851.py:16: DtypeWarning: Columns (5,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nodes = pd.read_csv(nodes_path, encoding='utf-8', delimiter=',')
C:\Users\relia\AppData\Local\Temp\ipykernel_5380\4245361851.py:17: DtypeWarning: Columns (10,11,12,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_roads = pd.read_csv(roads_path, encoding='utf-8', delimiter=',')


In [59]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')


df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Most_DC', 'Názov_mosta', 'ID_DC',
       'správcovské_číslo', 'Zemepisná_šírka', 'Zemepisná_dĺžka', 'Výška',
       'Druh_konštrukcie', 'Material', 'Normálna_zaťažiteľnosť',
       'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
       'Spôsob_stanovenia', 'Predmet_premostenia', 'Rok_postavenia',
       'Zaťažovacia_trieda_STN', 'Počet_otvorov', 'Dĺžka_premostenia',
       'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023'],
      

In [60]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
bridges_KSK = df_bridges_all[
    ((df_bridges_all["Správca_úseku"] == 'SC KSK - KS') |
     (df_bridges_all["Správca_úseku"] == 'SC KSK - SNV') |
     (df_bridges_all["Správca_úseku"] == 'SC KSK - MI') |
     (df_bridges_all["Správca_úseku"] == 'SC KSK - TV') |
     (df_bridges_all["Správca_úseku"] == 'SC KSK - RV'))
]

# Reindex the DataFrame
bridges_KSK.reset_index(drop=True, inplace=True)

bridges_KSK = bridges_KSK[bridges_KSK["n_2023"] != 0]

# Replace double space with single space in each item of the "Predmet_premostenia" column
bridges_KSK["Predmet_premostenia"] = bridges_KSK["Predmet_premostenia"].str.replace(
    "  ", " ")

# Display the filtered DataFrame
bridges_KSK.shape

(653, 51)

In [61]:
import geopandas as gpd
# Replace the file path with your actual file path
# file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # doma
file_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson' # robota


# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(file_path)
# Display the first few rows of the GeoDataFrame

In [62]:
KSK_survey_path = r"C:\Users\relia\Documents\GitHub\Bridges\08_SC_KE\DT\filtered_bridges.csv"
df_KSK_survey_path = pd.read_csv(
    KSK_survey_path, encoding='utf-8', delimiter=',')

In [63]:
df_KSK_survey_path.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Zemepisná_šírka',
       'Zemepisná_dĺžka', 'ÚSEK'],
      dtype='object')

In [64]:
df_KSK_survey_path["ÚSEK"] = df_KSK_survey_path["ÚSEK"].astype(str)
df_KSK_survey_path["ÚSEK"].value_counts()

ÚSEK
n       282
2048     10
2120      8
2080      8
2110      7
       ... 
2670      1
2650      1
2640      1
2630      1
5060      1
Name: count, Length: 167, dtype: int64

In [65]:
import numpy as np

# Replace "n" with NaN for now to preserve it during formatting
df_KSK_survey_path["ÚSEK"].replace("n", np.nan, inplace=True)

# Convert values to string and fill with leading zeroes
df_KSK_survey_path["ÚSEK"] = df_KSK_survey_path["ÚSEK"].astype(
    str).str.zfill(5)

# Replace NaN values back to "n"
df_KSK_survey_path["ÚSEK"].fillna("n", inplace=True)

# Replace "00nan" with "unknown"
df_KSK_survey_path["ÚSEK"].replace("00nan", "unknown", inplace=True)


# Print the updated value counts
df_KSK_survey_path["ÚSEK"].value_counts()

ÚSEK
unknown    282
02048       10
02120        8
02080        8
02110        7
          ... 
02670        1
02650        1
02640        1
02630        1
05060        1
Name: count, Length: 167, dtype: int64

In [66]:
# Convert values to string and fill with leading zeroes
df_survey["ÚSEK"] = df_survey["ÚSEK"].astype(
    str).str.zfill(5)

df_survey["ÚSEK"]

0       87020
1       87021
2       87022
3       87023
4       87024
        ...  
2751    94911
2752    94920
2753    95211
2754    94040
2755    94041
Name: ÚSEK, Length: 2756, dtype: object

In [67]:
bridges_KSK.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Most_DC', 'Názov_mosta', 'ID_DC',
       'správcovské_číslo', 'Zemepisná_šírka', 'Zemepisná_dĺžka', 'Výška',
       'Druh_konštrukcie', 'Material', 'Normálna_zaťažiteľnosť',
       'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
       'Spôsob_stanovenia', 'Predmet_premostenia', 'Rok_postavenia',
       'Zaťažovacia_trieda_STN', 'Počet_otvorov', 'Dĺžka_premostenia',
       'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023'],
      

In [68]:
df_survey.columns

Index(['ÚSEK', 'CESTA', 'OKRES', 'T', 'O', 'M', 'S'], dtype='object')

In [69]:
df_KSK_survey_path.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Zemepisná_šírka',
       'Zemepisná_dĺžka', 'ÚSEK'],
      dtype='object')

In [70]:
import pandas as pd

# Assuming bridges_KSK, df_KSK_survey_path, and df_survey are already loaded DataFrames

# Step 1: Merge bridges_KSK with df_KSK_survey_path
merged_data = pd.merge(bridges_KSK, df_KSK_survey_path[[
                       'ID_mosta', 'ÚSEK']], on='ID_mosta', how='left')

# Step 2: Merge the result with df_survey to get the traffic intensity "S"
final_data = pd.merge(
    merged_data, df_survey[['ÚSEK', 'S']], on='ÚSEK', how='left')

# Step 3: If there is no match for ÚSEK in df_survey, "S" will be NaN by default due to how='left' merge
bridges_KSK['S'] = final_data['S']

In [82]:
# Count the number of NaN values in the column 'S'
nan_count = bridges_KSK['S'].isna().sum()

# Print the number of NaN values
print("Number of NaNs in bridges_KSK['S']:",
      nan_count, bridges_KSK.shape, 653-284)

Number of NaNs in bridges_KSK['S']: 284 (653, 52) 369


In [ ]:
bridges_KSK

In [86]:
# Sorting bridges_KSK by the values in the column 'S' in descending order
sorted_bridges_KSK = bridges_KSK.sort_values(by='S', ascending=1)

# Display the sorted DataFrame
sorted_bridges_KSK[["ID_mosta","Názov_mosta", "S"]]

,ID_mosta,Názov_mosta,S
607,M4779,Most cez kanál Kalčava na štátnej hranici s MR,168.0
486,M7424,Most cez miestny potok - Turzovské kúpele,191.0
362,M6362,Most cez potok Okna pred obcou Blatná Polianka,194.0
363,M6141,Most cez potok v obci Blatná Polianka,194.0
364,M2637,Most cez Sobranecký potok za obcou Blatná Poli...,194.0
...,...,...,...
642,M614,566002- 004 Most cez potok v obci Inovce,NaN
643,M6794,566003- 002 Most cez potok na lúkach pred obco...,NaN
644,M5131,566003- 003 Most cez potok pred obcou Ruská By...,NaN
645,M4016,566003- 004 Most cez potok pred obcou Ruská By...,NaN
